In [ ]:
import numpy as np
import pandas as pd
import Utils
from smt.sampling_methods import LHS
import pickle
from smt.problems import Branin
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
from matplotlib.transforms import offset_copy
import random
from collections import namedtuple
plt.style.use('ggplot')

## Bayesian Optimization

In [ ]:
def bayesian_optimisation(n_iters, bounds, x0, alpha = 1e-5, epsilon = 1e-7):
    
    """ bayesian_optimisation
    Uses Gaussian Processes 
    Arguments:
    ----------
        n_iters: integer.
            Number of iterations to run the search algorithm.
        bounds: array-like, shape = [n_params, 2].
            Lower and upper bounds on the parameters of the function.
        x0: array-like, shape = [n_pre_samples, n_params].
            Array of initial points to sample the objective function. 
        alpha: double.
            Variance of the error term of the GP.
        epsilon: double.
            Precision tolerance for floats.
    """
    f_x = np.zeros(n_iters) 
    
    n_params = 10
    x_list = []
    y_list = []
    
    for params in x0:
        x_list.append(params)
        y_list.append(Utils.ten_dimensional(params))
        
    xp = np.array(x_list)
    yp = np.array(y_list)

    
    for n in range(n_iters):
        
        scaler = MinMaxScaler().fit(xp)
        
        kernel = Matern(nu = 3/2) 
        model = GaussianProcessRegressor(kernel = kernel, alpha = alpha, n_restarts_optimizer = 15, normalize_y = True, random_state = 0)
        model.fit(scaler.transform(xp),yp) #fit the model
        
        opt = Utils.find_robust_optimum (Model = model, Scaler = scaler) #find reference robust optimum
        
        #loc = Utils.find_robust_optimum_location (Model = model, Scaler = scaler)
        #noise = Utils.noise(Point = loc , Model = model, Scaler = scaler)
        #f_x[n] = Utils.branin(loc.reshape(1,-1) + noise.reshape(1,-1))
        f_x[n] = opt
        
        next_sample = Utils.sample_next_point (acquisition_func = Utils.expected_improvement, 
                                               Model = model, robust_optimum = opt, Scaler = scaler) #find next sample point by maximizing acquisiotn function

        
        if np.any(np.abs(next_sample - xp) <= epsilon):
            #print ('Iteration----'+str(n+1)+': Duplicate Sample Point')
            continue
        
        
        score = Utils.ten_dimensional (next_sample)

        # Update lists
        x_list.append(next_sample.ravel())
        y_list.append(score)
        
        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

        
        
    return xp, yp, opt, model, f_x

## 1. Set the Global Parameters

In [ ]:
n = 100
n_total =  200
dimensionality = 10
bounds = ((-0.8,0.8),(-0.8,0.8),(-0.8,0.8), (-0.8,0.8),
                                (-0.8,0.8),(-0.8,0.8), (-0.8,0.8),(-0.8,0.8),(-0.8,0.8),(-0.8,0.8))
xlimits = np.array([[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1],[-1,1]])
print ('Global Parameters Set..............')

## 2. Choose Initial Samples and run Bayesian Optimization

In [ ]:
runs = 5
res = []
for run in range(runs):
    print (run + 1)
    X = Utils.DOE(n_obs = n , xlimits = xlimits, random_state = run + 1 , criterion = 'm' )
    Temp = bayesian_optimisation( n_iters = n_total-n , bounds = bounds, x0 = X)
    res.append(Temp)
pickle.dump(res, open("res(1-5)---Ten-Dimensional---EIC-NL=2.p", "wb"))
from google.colab import files
files.download('res(1-5)---Ten-Dimensional---EIC-NL=2.p')

1
